<a href="https://colab.research.google.com/github/tanakt-hub/Test/blob/main/medBERTjp%EF%BC%8DLIME_v0_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 事前準備

## Mecab辞書のダウンロード

In [ ]:
import os

# MeCab & NEologd
!apt install mecab libmecab-dev mecab-ipadic-utf8 file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -a -y # 公式では -a オプションはついていないが多分必要
os.environ['MECABRC'] = "/etc/mecabrc" # 環境変数でmecabrcの場所を指定

# 万病辞書
# 最新版を使い場合はZipファイルをダウンロードして適用する：https://sociocom.naist.jp/j-meddic-for-mecab/
!wget http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic

import subprocess
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_dic_dir_path = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

# 万病辞書へのパス
manbyo_dic_path = 'MANBYO_201907_Dic-utf8.dic'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 11 newly installed, 0 to remove and 20 not upgraded.
Need to get 29.3 MB of archives.
After this operation, 282 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-up

## BERT事前学習モデルのダウンロード

3つのいずれかを指定
- UTHBERT: [UTH-BERT](https://ai-health.m.u-tokyo.ac.jp/home/research/uth-bert):
  - MeCab辞書：mecab-ipadic-neologd + J-MeDic (MANBYO_201907)
  - max_seq_length=512
- MEDBERT: [medBERTjp - MeCab-IPAdic-NEologd-JMeDic](https://github.com/ou-medinfo/medbertjp)
  - MeCab辞書：mecab-ipadic-neologd + J-MeDic (MANBYO_201907)
  - max_seq_length=128
- CLTOHOKU: [cl-tohoku/bert-base-japanese-whole-word-masking](https://github.com/cl-tohoku/bert-japanese)
  - MeCab辞書：mecab-ipadic-neologd
  - max_seq_length=512


In [ ]:
#BERT_MODEL = 'UTHBERT'
#BERT_MODEL = 'MEDBERT'
BERT_MODEL = 'CLTOHOKU'

In [ ]:
if BERT_MODEL == 'UTHBERT':
  !wget https://ai-health.m.u-tokyo.ac.jp/labweb/dl/uth_bert/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
  !unzip UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip

  model_name = 'UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K'
  MeCabDic = {"mecab_dic": None, "mecab_option": "-d " + neologd_dic_dir_path + " -u " + manbyo_dic_path}
  MaxSeqLen = 512

elif BERT_MODEL == 'MEDBERT':
  !wget https://github.com/ou-medinfo/medbertjp/releases/download/v0.1-minj/medBERTjp_L12_H768_A12_WWM_mecab-ipadic-neologd-jmedic.zip
  !unzip medBERTjp_L12_H768_A12_WWM_mecab-ipadic-neologd-jmedic.zip

  model_name = 'medBERTjp_L12_H768_A12_WWM_mecab-ipadic-neologd-jmedic'
  MeCabDic = {"mecab_dic": None, "mecab_option": "-d " + neologd_dic_dir_path + " -u " + manbyo_dic_path}
  MaxSeqLen = 128

elif BERT_MODEL == 'CLTOHOKU':
  model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
#  MeCabDic = {"mecab_dic": None, "mecab_option": "-d " + neologd_dic_dir_path + " -u " + manbyo_dic_path}
  MeCabDic = {"mecab_dic": None, "mecab_option": "-d " + neologd_dic_dir_path}
  MaxSeqLen = 512

else:
  print('ERROR: 有効なBERTモデルが定義されていません')


## pip install

In [ ]:
!pip install transformers
!pip install mecab-python3 fugashi #ipadicは使わないため準備不要
!pip install lime
!pip install jaconv neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## import

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import  AdamW, BertJapaneseTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, random_split

from tqdm.notebook import tqdm
from IPython.display import display, HTML
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from lime.lime_text import LimeTextExplainer

import jaconv
import unicodedata
import neologdn
import re

# デバイス設定
device = torch.device("cuda")

## 前処理関数の定義

In [ ]:
def preprocess(text, model):
  text = neologdn.normalize(text)
  text = unicodedata.normalize("NFKC", text)

  #UTH-BERTは正規化の後に以下処理が必要
  if model == 'UTHBERT':
    text = re.sub(r'(\d)([,])(\d+)', r'\1\3', text) # 数字中の区切りカンマを削除
    text = text.replace(",", "、")  # 文中のカンマを削除
    text = text.replace("，", "、") # 文中のカンマを削除
    text = (jaconv.h2z(text, kana=True, digit=True, ascii=True))  # 全角文字変換
    text = text.replace("\u3000", "") # 全角スペース削除

  return text

## Transformersに送る辞書と分類設定


In [ ]:
# トークナイザの設定
tokenizer = BertJapaneseTokenizer.from_pretrained(
    model_name,
    word_tokenizer_type = "mecab",
    mecab_kwargs = MeCabDic
    )

In [ ]:
# BertForSequenceClassificationに事前学習モデルをロード
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels = 2,                # Binary classification
    output_attentions = False,     # Attentionの出力
    output_hidden_states = False,  # 隠れ層の出力
    )

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [ ]:
# モデルをGPUへ転送
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

# 学習データの準備

## 学習パラメータの設定

In [ ]:
# 学習パラメータの設定
# バッチサイズの設定
BATCH_SIZE = 64

# Learning Rateno設定
LEAENING_RATE = 1e-6

# エポック数の設定
N_EPOCHS = 15

## 学習用データのトークナイズ

In [ ]:
# 学習用データ取得
df = pd.read_table("https://raw.githubusercontent.com/tanakt-hub/Test/main/data/Label-y_v3.txt")

text = df.text
label = df.flg

# 最大トークン数の確認
# ロードしたモデルの入力最大値を超える場合は切り捨て処理を行うため注意
# medBERTjp = 128, UTH-BERT = 512, cl-tohoku/wwm = 512

model_seq_len = MaxSeqLen

max_tk = 0
for i, chktoken in enumerate(text):
  tk = tokenizer.tokenize(preprocess(chktoken))
  if len(tk) > max_tk:
    max_tk = len(tk)
    id = i

max_len = max_tk + 2 if max_tk + 2 < model_seq_len else model_seq_len

tokchk = tokenizer.encode_plus(
    preprocess(text[id], BERT_MODEL), # 入力値を前処理関数で処理
    add_special_tokens = True,        # スペシャルトークンの追加
    truncation = True,                # モデル定義長を超える場合の切り捨て
    max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
    padding = 'max_length',           # 最大長までPADDING
    return_overflowing_tokens = True, # 切り捨てられたトークンを返す
    num_truncated_tokens = True       # 切り捨てられたトークン数を返す
    )

print("最大トークン数:", max_tk)
print("*** 最大トークン数に分割されるテキスト ***")
print("  ", text[id])
print("*** BERTに入力されるテキスト ***")
print("  ", tokenizer.decode(tokchk['input_ids']))
print("*** 切り捨てられたテキスト ***")
print("  ", tokenizer.decode(tokchk['overflowing_tokens']))


Keyword arguments {'num_truncated_tokens': True} not recognized.


最大トークン数: 188
*** 最大トークン数に分割されるテキスト ***
   血液検査では、正球性貧血（red cell count 2.40 x 10^6/uL, reference range 3.86-4.92 x 10^6/uL; hemoglobin 7.8 g/dL, 11.6-14.8 g/dL; hematocrit 22.8%, 35.1%-44.4%)と、正常の血小板数(159 x 10^3/uL, 158-348 x 10^3/uL), prothrombin time (10.9 s, 9.5-13.5 s), activated partial thromboplastin time (25.9 s, 25.0-38.0 s)と判明した。
*** BERTに入力されるテキスト ***
   [CLS] 血液検査 で は 、 正 球 性 貧血 ( red cell count 2. 40 x 10 [UNK] 6 / uL, reference range 3. 86 -4. 92 x 10 [UNK] 6 / uL ; hemoglobin 7. 8 g / dL, 11. 6 -1 4. 8 g / dL ; hematocrit 22.8%, 35.1% -4 4.4% ) と 、 正常 の 血小板 数 ( 159 x 10 [UNK] 3 / uL, 158 -3 48 x 10 [UNK] 3 / uL ), prothrombin time ( 10. 9 s, 9. 5 - 13. 5 s ), activated partial thromboplastin time ( 25. 9 s, 25. 0 -3 8. 0 s ) と 判明 し た 。 [SEP]
*** 切り捨てられたテキスト ***
   


In [ ]:
# トークナイズ処理
# 必要なToken IDとAttentionマスクを取得

token_ids = []
attention_masks = []

for t in text:
  tknzd = tokenizer.encode_plus(
      preprocess(t, BERT_MODEL),        # 入力値を前処理関数で処理
      add_special_tokens = True,        # スペシャルトークンの追加
      truncation = True,                # モデル定義長を超える場合の切り捨て
      max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
      padding = 'max_length'            # 最大長までPADDING
      )
  token_ids.append(tknzd['input_ids'])
  attention_masks.append(tknzd['attention_mask'])

# tensor型に変換
token_ids_t = torch.tensor(token_ids)
attention_masks_t = torch.tensor(attention_masks)
labels_t = torch.tensor(label)

x = 0
print(tokenizer.tokenize(text[x]))
print(token_ids_t[x])
print(attention_masks_t[x])
print(labels_t[x])

['両側', 'の', '肺炎', 'と', '呼吸', '##不全', 'を', '指摘', 'さ', 'れ', '入院', '。']
tensor([    2, 10395,     5, 14207,    13,  9489, 18956,    11,  2353,    26,
           20,  8305,     8,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,   

## データセットとデータローダーの作成

In [ ]:
# 全データをデータセット化
dataset = TensorDataset(token_ids_t, attention_masks_t, labels_t)

# 学習用と検証用にデータ分割

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

SEED = 1111 # Random seedの指定
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(SEED))

print('学習データ数：', len(train_dataset))
print('検証データ数: ', len(val_dataset))

学習データ数： 2528
検証データ数:  633


In [ ]:
# データローダーの作成
train_dataloader = DataLoader(
    train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,     # ランダムで取得するか否か
    drop_last = True    # バッチ数に満たないラストデータを落とすか否か
    )
val_dataloader = DataLoader(
    val_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    drop_last = False
    )


# Fine-tuningの実行

In [ ]:
# 最適化手法の設定
optimizer = torch.optim.AdamW(model.parameters(), lr=LEAENING_RATE)

# 訓練パート関数の定義
def train(train_dataloader, model, optimizer, device, tqdm):
  train_losses = []
  model.train()     # 訓練モード
  optimizer.zero_grad()
  for n_iter, d in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
    input_ids_t_ = d[0].to(device)  # input_ids_t
    attention_masks_t_ = d[1].to(device)  #"attention_masks_t
    labels_t_ = d[2].to(device) # labels_t

    outputs = model(
        input_ids_t_, 
        attention_mask = attention_masks_t_, 
        labels = labels_t_,
          token_type_ids=None
          )
    loss = outputs.loss # BertForSequenceClassificationで損失関数CrossEntropyLossの結果を返す
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 勾配クリッピング

    optimizer.step()
    optimizer.zero_grad()

    train_losses.append(loss.item())
  return train_losses

# 検証パート関数の定義
def validation(val_dataloader, model, device, tqdm):
  val_losses = []
  logits     = []
  labels     = []
  inputs     = []

  model.eval()    # 検証モード
  for n_iter, d in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
    input_ids_t_ = d[0].to(device)  # input_ids_t
    attention_masks_t_ = d[1].to(device)  #"attention_masks_t
    labels_t_ = d[2].to(device) # labels_t

    with torch.no_grad():
      outputs = model(
          input_ids_t_, 
          attention_mask = attention_masks_t_, 
          labels = labels_t_,
          token_type_ids=None
            )

    loss = outputs.loss # BertForSequenceClassificationで損失関数CrossEntropyLossの結果を返す(single_label_classificationのため)
    val_losses.append(loss.item())
    
    logits += outputs.logits.sigmoid().cpu().tolist()
    labels += labels_t_.cpu().tolist()
    inputs += input_ids_t_.cpu().tolist()

  # Predictionの結果をDataFrameで返す
  val_res = pd.DataFrame(logits, columns=['logit0', 'logit1'])
  val_pred = np.argmax(val_res.values, axis=1).tolist()
  val_res['label'] = labels
  val_res['pred']  = val_pred
  val_res['input_ids']  = inputs
  
  return val_losses, val_res

In [ ]:
# ミニバッチごとのLossを格納する変数を定義
train_losses = []
val_losses = []

# 学習の実施
for epoch in range(N_EPOCHS):
  print('***********************')
  print('*** epoch', epoch, 'started')
  print('***********************')

  train_ = train(train_dataloader, model, optimizer, device, tqdm)
  loss, val_res = validation(val_dataloader, model, device, tqdm)

  cm = confusion_matrix(val_res['label'].tolist(), val_res['pred'].tolist())
  cm_df = pd.DataFrame(cm,columns=['Predicted Neg', 'Predicted Pos'], index=['Actual Neg', 'Actual Pos'])

  display(cm_df)
  print('epoch', epoch, 'total loss :', sum(loss), '\n')

  train_losses += train_
  val_losses += loss

***********************
*** epoch 0 started
***********************


  0%|          | 0/39 [00:00<?, ?it/s]

RuntimeError: ignored

# 学習結果の確認

In [ ]:
# 学習時のミニバッチごとのLoss
# 横軸は Epoch数 * Epoch内のミニバッチ数が最大値
plt.plot(train_losses)

In [ ]:
# 検証時のミニバッチごとのLoss
# 横軸は Epoch数 * Epoch内のミニバッチ数が最大値
# データローダーでShuffle設定していない場合はパターンのあるグラフが出る
plt.plot(val_losses)

In [ ]:
# 教師ラベルと異なる予測となった一覧
val_res['Text'] = [t.strip('[CLS] [SEP] [PAD]') for t in tokenizer.batch_decode(val_res['input_ids'])]
Errors = val_res.query('label!=pred')
Errors

# LIMEによる可視化

 - LIME (TextExplainer)
   - テキストを与えると、一部のトークンをマスクした文章をnum_samples分つくる
   - マスクした文章をpredictorに通した戻り値から、判定への寄与度を近似する
 - predictor関数
   - predictorの入力値には、num_samples分の文章がリストで与えられる
   - 各文章はsplitterで分割された各トークンの一部をmask_stringで置き換えたもの
   - Fine-tuningしたBERTモデルに通すことでlogitsを得る
 - LimeTextExplainerの引数
   - split_expression: デフォルトの区切り値が\Wなので\sで上書き
   - bow: BERTモデルではトークンの位置情報も影響するためFalse
   - mask_string: BERTのUNKトークンを設定
 - explain_instance
   - Fine-tuningに使用したTokenizerで分かち書き（区切り値\s）したものを入力テキストとする

In [ ]:
def predictor(text):
  tok = tokenizer.batch_encode_plus(
    text,
    padding=True
    )
  input_ids = torch.tensor(tok['input_ids']).to(device)
  attention_mask = torch.tensor(tok['attention_mask']).to(device)
  token_type_ids = torch.tensor(tok['token_type_ids']).to(device)

  with torch.no_grad():
    output = model(input_ids, attention_mask, token_type_ids)

  probas = output.logits.sigmoid().cpu().numpy()
  return probas

explainer = LimeTextExplainer(class_names=['Neg', 'Pos'], split_expression=r'\s', bow=False, mask_string='[UNK]')

In [ ]:
# ↑のセルまで実行完了後はこのセルのみでテスト可能
# ダブルクォーテーションの中の文章を好きに入れ替えてCtrl+Enterで実行結果が更新

txt = "組織脳脊髄液検査では異常が無かったが、細菌培養及びウイルス検査は異常を認めた。"

str_to_predict = tokenizer.tokenize(preprocess(txt, BERT_MODEL))
txtw = ' '.join(str_to_predict)

exp = explainer.explain_instance(txtw, predictor, num_features=10, num_samples=1000)
exp.show_in_notebook()

# 未知データの分類

ラベルの無いデータ群に対する予測ラベルの付与

In [ ]:
# ラベル無しデータ取得
df = pd.read_table("https://raw.githubusercontent.com/tanakt-hub/Test/main/data/Label-n_v3.txt")

text = df.text

# 最大トークン数の確認
# ロードしたモデルの入力最大値を超える場合は切り捨て処理を行うため注意
# medBERTjp = 128, UTH-BERT = 512, cl-tohoku/wwm = 512

model_seq_len = MaxSeqLen

max_tk = 0
for i, chktoken in enumerate(text):
  tk = tokenizer.tokenize(preprocess(chktoken))
  if len(tk) > max_tk:
    max_tk = len(tk)
    id = i

max_len = max_tk + 2 if max_tk + 2 < model_seq_len else model_seq_len

tokchk = tokenizer.encode_plus(
    preprocess(text[id], BERT_MODEL), # 入力値を前処理関数で処理
    add_special_tokens = True,        # スペシャルトークンの追加
    truncation = True,                # モデル定義長を超える場合の切り捨て
    max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
    padding = 'max_length',           # 最大長までPADDING
    return_overflowing_tokens = True, # 切り捨てられたトークンを返す
    num_truncated_tokens = True       # 切り捨てられたトークン数を返す
    )

print("最大トークン数:", max_tk)
print("*** 最大トークン数に分割されるテキスト ***")
print("  ", text[id])
print("*** BERTに入力されるテキスト ***")
print("  ", tokenizer.decode(tokchk['input_ids']))
print("*** 切り捨てられたテキスト ***")
print("  ", tokenizer.decode(tokchk['overflowing_tokens']))


In [ ]:
# トークナイズ処理
# 必要なToken IDとAttentionマスクを取得

token_ids = []
attention_masks = []

for t in text:
  tknzd = tokenizer.encode_plus(
      preprocess(t, BERT_MODEL),        # 入力値を前処理関数で処理
      add_special_tokens = True,        # スペシャルトークンの追加
      truncation = True,                # モデル定義長を超える場合の切り捨て
      max_length = max_len,             # モデル定義内の場合は入力値の最大長に再定義
      padding = 'max_length'            # 最大長までPADDING
      )
  token_ids.append(tknzd['input_ids'])
  attention_masks.append(tknzd['attention_mask'])

# tensor型に変換
token_ids_t = torch.tensor(token_ids)
attention_masks_t = torch.tensor(attention_masks)

x = 0
print(tokenizer.tokenize(text[x]))
print(token_ids_t[x])
print(attention_masks_t[x])


In [ ]:
# テストパート関数の定義
def eval(eval_dataloader, model, device, tqdm):
  logits     = []
  inputs     = []

  model.eval()    # 検証モード
  for n_iter, d in tqdm(enumerate(eval_dataloader), total=len(eval_dataloader)):
    input_ids_t_ = d[0].to(device)  # input_ids_t
    attention_masks_t_ = d[1].to(device)  #"attention_masks_t

    with torch.no_grad():
      outputs = model(
          input_ids_t_, 
          attention_mask = attention_masks_t_, 
          token_type_ids=None
            )

    
    logits += outputs.logits.sigmoid().cpu().tolist()
    inputs += input_ids_t_.cpu().tolist()
  
  eval_res = pd.DataFrame(logits, columns=['logit0', 'logit1'])
  eval_pred = np.argmax(eval_res.values, axis=1).tolist()
  eval_res['pred']  = eval_pred
  eval_res['input_ids']  = inputs

  return eval_res

In [ ]:
eval_dataset = TensorDataset(token_ids_t, attention_masks_t)
eval_dataloader = DataLoader(
    eval_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    drop_last = False
    )

#out = eval(eval_dataloader, model, device, tqdm)

In [ ]:
out['Text'] = [t.strip('[CLS] [SEP] [PAD]') for t in tokenizer.batch_decode(out['input_ids'])]
out['text1'] = text
out